## <font color=darkorange> Multi-layer Perceptron & Convolutional neural networks (CNN)</font>

### <font color=darkred> MLP for regression</font>

In [ ]:
# ignore warnings for better clarity (may not be the best thing to do)...
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from random import randint
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import activations
import numpy as np
import matplotlib.pyplot as plt
#from keras import backend as K
# ignore warnings for better clarity (may not be the best thing to do)...
import warnings
warnings.filterwarnings('ignore')
from keras.datasets import cifar100

In [ ]:
from keras.datasets import mnist, boston_housing
from sklearn.preprocessing import MinMaxScaler

Dataset taken from the StatLib library which is maintained at Carnegie Mellon University.

Samples contain 13 attributes of houses at different locations around the Boston suburbs in the late 1970s. Targets are the median values of the houses at a location (in k$).
https://keras.io/datasets/#boston-housing-price-regression-dataset

In [ ]:
(x_train, y_train), (x_test, y_test) = boston_housing.load_data()
scaler_x = MinMaxScaler()
scaler_x.fit(x_train)
x_train = scaler_x.transform(x_train)
x_test = scaler_x.transform(x_test)
scaler_y = MinMaxScaler()
scaler_y.fit(y_train.reshape((-1, 1)))
y_train = scaler_y.transform(y_train.reshape((-1, 1)))
y_test = scaler_y.transform(y_test.reshape((-1, 1)))

In [ ]:
print(x_train.shape, y_train.shape)

In [ ]:
layer_in = Dense(units = 64, activation = "relu", input_dim = x_train.shape[1])
layer_h1 = Dense(units = 64, activation = "relu")
layer_h2 = Dense(units = 64, activation = "relu")
layer_out = Dense(units = y_train.shape[1], activation = "linear")

In [ ]:
model = Sequential()
model.add(layer_in)
model.add(layer_h1)
model.add(layer_h2)
model.add(layer_out)

model.compile(optimizer="sgd", loss="mse")
model.summary()

In [ ]:
batch_size = 32
epochs = 25
history = model.fit(x_train, y_train,
                         batch_size=batch_size,
                         epochs=epochs,
                         verbose=1,
                         validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)

In [ ]:
plt.figure(figsize=(7, 5))
plt.plot(history.epoch, history.history['loss'], lw=1, label='Train')
plt.plot(history.epoch, history.history['val_loss'], lw=1, label='Test')
plt.legend(fontsize=14)
plt.title('Loss for FFNN + linear regression', fontsize=16)
plt.xlabel('Epoch', fontsize=14)
plt.ylabel('Loss', fontsize=14)
plt.tight_layout()

#### <font color=darkred> Optimization methods for neural networks</font>

``The ADAptive GRADient`` algorithm introduced by http://www.jmlr.org/papers/volume12/duchi11a/duchi11a.pdf, (Duchi et al. 2011), starts from $w^{(0)}$ and uses a learning rate $\eta > 0$ and a momentum $\alpha$ and defines, for all $k\geqslant 0$ and all $j \in\{1, \ldots , d\}$, 
$$
w_j^{(k+1)} \gets w_j^{(k)}  - \frac{\eta}{\sqrt{\sum_{\tau=1}^k (\nabla f (w^{(\tau)}))_j^2}} (\nabla f(w^{(k)}))_j\,.
$$
The rationale of this method is that different rates are used for all coordinates which is crucial for neural networks in which gradient at different layers can be of different order of magnitude. It is proved in https://arxiv.org/pdf/1806.01811.pdf (Ward et al., 2018) that AdaGrad achieves the same convergence rate as gradient descent with optimal fixed stepsize up to a $\log$ factor. The adaptive step size grows with the inverse of the gradient magnitudes, so that large gradients have small learning rates and small gradients have
large learning rates.

``AdaDelta`` Introduced in https://arxiv.org/pdf/1212.5701.pdf (Zeiler, 2012) to reduce the sensitivity to initial conditions of AdaGrad. Indeed, if the initial gradients are large, the learning rates of AdaGrad will be low for all updates which can be overcome by increasing $\eta$, but making the AdaGrad method highly sensitive to the choice of $\eta$.  


``RMSprop optimizer`` Unpublished method, from the course of http://www.cs.toronto.edu/~tijmen/csc321/slides/lecture_slides_lec6.pdf (Geoff Hinton).


``ADAM: Adaptive moment estimation`` Introduced in https://arxiv.org/pdf/1412.6980.pdf (Kingma et al., 2014) and considered as the state of the art to otpimize neural networks, the ADAM procedure update the parameter estimate as follows. Starting from $m_0 = 0$ and $v_0=0$ and choosing $\beta_1, \beta_2, \eta, \varepsilon
 \in (0,1)$, compute first and second moment estimate 
$$
m_k = \beta_1 m_{k-1} + (1 - \beta_1) \nabla f (w^{(k)}) \quad  \mathrm{and} \quad v_k = \beta_2 v_{k-1} + (1 - \beta_2) (\nabla f (w^{(k)}))^2\,,
$$
then, compute the correction terms
$$
\hat{m}_k = \frac{m_k}{1 - \beta_1^k} \quad \hat{v}_k = \frac{v_k}{1 - \beta_2^k}\,,
$$
and update the parameter estimate with
$$
w^{(k+1)} = w^{(k)} - \frac{\eta}{\sqrt{\hat{v}_k}+ \varepsilon} \hat{m}_k\,.
$$
First convergence results can be found in https://arxiv.org/pdf/1412.6980.pdf (Kingma et al., 2014) and examples where ADAM algorithm does not converge to the optimum are given in https://openreview.net/pdf?id=ryQu7f-RZ (Reddi et al., 2018). Recent analysis by https://arxiv.org/abs/1810.02263 (Barakat et al., 2018).

In [ ]:
model = Sequential()
model.add(layer_in)
model.add(layer_h1)
model.add(layer_h2)
model.add(layer_out)

model.compile(optimizer="adam", loss="mse")
model.summary()

In [ ]:
batch_size = 32
epochs = 25
history = model.fit(x_train, y_train,
                         batch_size=batch_size,
                         epochs=epochs,
                         verbose=1,
                         validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
y_pred_ffnn = model.predict(x_test)

In [ ]:
plt.figure(figsize=(7, 5))
plt.plot(history.epoch, history.history['loss'], lw=1, label='Train')
plt.plot(history.epoch, history.history['val_loss'], lw=1, label='Test')
plt.legend(fontsize=14)
plt.title('Loss for FFNN + linear regression', fontsize=16)
plt.xlabel('Epoch', fontsize=14)
plt.ylabel('Loss', fontsize=14)
plt.tight_layout()

#### <font color=darkred> Cross-validation neural networks</font>

In [ ]:
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import GridSearchCV

In [ ]:
def ffnn_model(nb_layer = 3, nb_neuron = 10):
    layer_in = Dense(units = nb_neuron, activation="relu", input_dim = x_train.shape[1])
    model = Sequential()
    model.add(layer_in)
    for i in range(nb_layer):
        hidden_layer = Dense(units = nb_neuron, activation="relu")
        model.add(hidden_layer)
    
    layer_out = Dense(units = y_train.shape[1], activation="relu")
    model.add(layer_out)
  
    model.compile(optimizer="adam", loss="mse")
    return model


parameters = {'nb_layer': [2, 3], 
              'nb_neuron': [32, 64, 128],
              'batch_size': [16, 32]}

keras_model = KerasRegressor(build_fn = ffnn_model, epochs = 25)

clf = GridSearchCV(keras_model, parameters, cv = 3)
clf.fit(x_train, y_train)

In [ ]:
y_pred = clf.predict(x_test)
plt.figure(figsize=(12, 12))
plt.plot(range(x_test.shape[0]), y_pred, lw=1, label='Prediction after cross validation')
plt.plot(range(x_test.shape[0]), y_pred_ffnn, lw=1, label='Prediction without cross validation')
plt.plot(range(x_test.shape[0]), y_test, lw=1, label='True output')
plt.legend(fontsize=14)
plt.title('Prediction', fontsize=16)
plt.xlabel('Test data input', fontsize=14)
plt.ylabel('Test data output', fontsize=14)
plt.tight_layout()

### <font color=darkred> Convolution for image classification</font>

The CIFAR-10 and CIFAR-100 are labeled subsets of the 80 million tiny images dataset. They were collected by Alex Krizhevsky, Vinod Nair, and Geoffrey Hinton: https://www.cs.toronto.edu/~kriz/cifar.html
The dataset contains 60000 32x32x3 colour images divided in 100 classes.

In [ ]:
# Number of classes
num_classes = 100
# input image dimensions
img_rows, img_cols = 32, 32

# cifar100 data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = cifar100.load_data()

input_shape = (img_rows, img_cols, 3)

print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

print('input dimension:',x_train.shape[1::])

print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

In [ ]:
# normalize the input data to obtain entries in (0,1)
x_train = x_train/255
x_test  = x_test/255

In [ ]:
# display one input data at random
plt.imshow(x_train[randint(0, x_train.shape[0])])

In [ ]:
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
y_train[:1]

#### <font color=darkred> Softmax regression </font>
Logistic regression can be extended to classify data in more than two groups. Softmax regression provides a model for the probability that an input $x$ is associated with each group.  It is assumed that the probability to belong to the class $k\in\{1,\ldots,M\}$ can be expressed by 
\begin{equation*}
\mathbb{P}(Y = k| X) = \frac{\exp(\langle w_k,X \rangle + b_k)}{\sum_{\ell=1}^{M}\exp(\langle w_\ell,X \rangle + b_\ell)} = p_k(X)\,,
\end{equation*}
where $w_\ell \in \mathbb{R}^d$ and $b_\ell$  are model `weights` and `intercepts` for each class.


To estimate these unknown parameters, a maximum likelihood approach is used as in the logistic regression setting. In this case, the loss function is given by the negative log-likelihood (see also the section on gradient based method).

In [ ]:
# Create a model prone to add layers sequentially
model = Sequential()
# flatten the data replaces 32 * 32 * 3 matrices by a 3072 dimensional vector
# This is always necessary before a fully-connected layer (Dense object)
model.add(Flatten(input_shape=input_shape, name='flatten'))
# add one dense (fully connected layer) with softmax activation function
# As it is the first layer, the input size is mandatory
model.add(Dense(num_classes, activation='softmax', name='dense_softmax'))

# "compile" this model, 
model.compile(
    # specify the loss as the cross-entropy i.e. the negative loglikelihood.
    loss=keras.losses.categorical_crossentropy,
    # choose the gradient based method to estimate the parameters
    # see https://keras.io/optimizers/ to have an overview of the different options
    # see also section 2 on gradient based methods.
    optimizer=keras.optimizers.Adagrad(),
    # metric to monitor on the test data
    metrics=['accuracy']
)
model.summary()

In [ ]:
# number of data used for each update of the parameter (each gradient computation)
batch_size = 64
# number of times data are scanned
epochs = 25
# train the model, i.e. estimate unknown parameters by minimizing the loss function using a gradient descent algorithm (here Adagrad).
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
plt.figure(figsize=(7, 5))
plt.plot(history.epoch, history.history['acc'], lw=1, label='Train')
plt.plot(history.epoch, history.history['val_acc'], lw=1, label='Test')
plt.legend(fontsize=14)
plt.title('Accuracy of softmax regression', fontsize=16)
plt.xlabel('Epoch', fontsize=14)
plt.ylabel('Accuracy', fontsize=14)
plt.tight_layout()

#### <font color=darkred> Feed-Forward Neural Network (FFNN) or multilayer Perceptron (MLP)</font>
The softmax regression of the previous section is a linear model, with 307300 parameters.  It might be too simple for our classification task.  The idea underlying neural networks is to have successive "neurons" performing a linear transformation of the input data (depending on a weight matrix and a bias vector) followed by an activation function to design more flexible models with additional parameters.

In [ ]:
# Create the graph for a fully connected feed-forward neural network with one hidden layer 
# with 256 units and a relu activation function. 
model_ffnn = Sequential()

model_ffnn.add(Flatten(input_shape=input_shape))

model_ffnn.add(Dense(256, activation='relu'))

model_ffnn.add(Dense(num_classes, activation='softmax'))

model_ffnn.compile(
    loss=keras.losses.categorical_crossentropy,
    optimizer=keras.optimizers.Adagrad(),
    metrics=['accuracy']
)

model_ffnn.summary()

In this model the input data $X$ lies in $\mathbb{R}^d$ with $d = 3072$.

A hidden layer is built in $\mathbb{R}^h$ with $h = 256$.

\begin{align*}
z^\mathrm{hid}(X) &= W^hX+b^h\,,\\
h(X) &= \mathrm{Relu}(z^\mathrm{hid}(X))\,.
\end{align*}

$W^h\in\mathbb{R}^{hxd}$, $b^h\in\mathbb{R}^h$, $h(X)\in\mathbb{R}^h$ and for all $1\leqslant j\leqslant h$, $h(X)_j = \mathrm{Max}(0,z^\mathrm{hid}(X)_j)$. 

The output layer is built in $\mathbb{R}^M$ with $M = 100$.

\begin{align*}
z^\mathrm{out}(X) &= W^oX+b^o\,,\\
f_{\theta}(X) &= \mathrm{Softmax}(z^\mathrm{out}(X))\,.
\end{align*}

$W^o\in\mathbb{R}^{Mxh}$, $b^o\in\mathbb{R}^o$. 

$\theta = (W^h,b^h,W^o,b^o)$.

$f_{\theta}(X)$ is a vector in $\mathbb{R}^M$ where each entry is the probability that $X$  belongs to the corresponding class.

In [ ]:
batch_size = 64
epochs = 25
history = model_ffnn.fit(x_train, y_train,
                         batch_size=batch_size,
                         epochs=epochs,
                         verbose=1,
                         validation_data=(x_test, y_test))
score = model_ffnn.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
plt.figure(figsize=(7, 5))
plt.plot(history.epoch, history.history['acc'], lw=1, label='Train')
plt.plot(history.epoch, history.history['val_acc'], lw=1, label='Test')
plt.legend(fontsize=14)
plt.title('Accuracy of softmax regression', fontsize=16)
plt.xlabel('Epoch', fontsize=14)
plt.ylabel('Accuracy', fontsize=14)
plt.tight_layout()

The number of parameters is much larger than in the softmax setting while the performance only slightly improves. See next course on `Convolutional networks to provide models more suitable to image data`...

<font color=darkred> Use cross-validation to find the best values of $h$ for instance in $\{32,64,128,256,512\}$...</font>

### <font color=darkred> Convolutional Neural Networks </font>

In practice, increasing the size of hidden layers is not very effective. 
It is often a better idea to add more layers. 
Intuitively, if the observations have a hierarchical structure, adding more layers can be interpreted as a way to learn more levels of abstraction.  For example, in object detection tasks, it is easier to express shapes from edges and objects from shapes, than to express objects from pixels.  Thus, a good design should try to exploit this hierarchy.

In particular cases, such as grid-like data (time series, images), the observations can depend on a pattern which can be associated with different locations of the data. 
For example, an object can be in the middle or the left of the picture. 
Thus, the model has to be translation invariant: it is easier to learn how to recognize an object independently of its location. 

When two inputs might contain the same kind of information, then it is useful to share their weights and estimate the weights jointly for those inputs to learn statistical invariants (things that don't change much on average across time or space). 
Using this concept on images leads to convolutional neural networks (CNNs), on text, it results on recurrent neural networks (RNNs).  When using CNNs, weights are set to a small kernel that is used to perform a convolution across the image.

In the previous example, the step ``model_ffnn.add(Flatten(input_shape=input_shape))`` destroys the spatial organization of the input but is mandatory before the fully connected step: ``model_ffnn.add(Dense(128, activation='relu'))``. The spatial organization of the input may be preserved before using Flatten by convolution steps such as ``model_cnn.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))``. This layer has 32 outputs, each of which depending on a 3*3 weight matrix.

Each output neuron of the next layer is computed as follows (image taken from https://adventuresinmachinelearning.com/convolutional-neural-networks-tutorial-tensorflow/) before computing an elementwise RELU function:
<img src="conv.PNG">

The convolution maps patches of the input image, combined with the convolution kernel, for example
$$
\text{output} = \text{ReLU}(\text{patch} \times W + b)\,.
$$

Performing the convolution between the image and the weight matrix consists in moving the kernel across the image, and to produce an output for each patch. The way you move across the image is defined by two parameters:

- ``Stride``: the stride is the number of pixels you are shifting each time you move your kernel during the convolution.
- ``Padding``: defines what happens when the kernel reaches a border of the image when doing the convolution. 
"Valid" padding means that you stop at the edge, while "Same" padding allows to go off the edge and pad with zeros so that the width and the height of the output and input tensors are the same.

A classical approach is to use a stride of 1 and to combine theoutputs being in some neighborhood. Such an operation combining elements of a tensor is called ``pooling``: ``model_cnn.add(MaxPooling2D(pool_size=(2, 2)))``. Neighborhoods are define by the pooling window dimension (width x height) and the strides you use when moving this window across the image.  Again, a sliding window is applied along the input to produce the output, the most common function applied on each window being the ma function: it aggregates several outputs in a neighborhood $N$ using a max operation: 
$$
\text{output}'_i = \max_{j \in N}\text{output}_j, \quad i \in N.
$$
The formulas to compute the size of the ouput tensor are the same as for convolution padding and striding: (image taken from https://adventuresinmachinelearning.com/convolutional-neural-networks-tutorial-tensorflow/):
<img src="maxpool.PNG">

In [ ]:
# Convolutional layer with 32 filters and 3 * 3 kernel sizes and 'relu' activation (use the `Conv2D` object)
# Convolutional layer with 64 filters and 3 * 3 kernel sizes and 'relu' activation (use the `Conv2D` object)
# Max pooling with pool size 2 * 2 (use the `MaxPooling2D` object)
# Dropout with probability 0.25 (use the `Dropout` object)
# Dense layer with 128 units with relu activation
# Dropout with probability 0.5
# Dense output layer with softmax activation

model_cnn = Sequential()

model_cnn.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
model_cnn.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model_cnn.add(MaxPooling2D(pool_size=(2, 2)))
model_cnn.add(Dropout(0.25))
model_cnn.add(Flatten())
model_cnn.add(Dense(128, activation='relu'))
model_cnn.add(Dropout(0.5))
model_cnn.add(Dense(num_classes, activation='softmax'))

model_cnn.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=keras.optimizers.Adadelta(),
                  metrics=['accuracy'])

model_cnn.summary()

In [ ]:
batch_size = 64
epochs = 25

# Run the train
history = model_cnn.fit(x_train, y_train,
                        batch_size=batch_size,
                        epochs=epochs,
                        verbose=1,
                        validation_data=(x_test, y_test))
score = model_cnn.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
plt.figure(figsize=(7, 5))
plt.plot(history.epoch, history.history['acc'], lw=3, label='Training')
plt.plot(history.epoch, history.history['val_acc'], lw=3, label='Testing')
plt.legend(fontsize=14)
plt.title('Accuracy of softmax regression', fontsize=16)
plt.xlabel('Epoch', fontsize=14)
plt.ylabel('Accuracy', fontsize=14)
plt.tight_layout()